### MERIT-Hydro–Derived River Network For South Saskachewan River

**Description**  
The regional river network used in this study was derived directly from the **MERIT Hydro** digital elevation model (DEM) and is **independent of the MERIT-Basins product**. The river network was generated specifically for a study region in **southern Saskatchewan, Canada**, centered around **Lake Diefenbaker and the Gardiner Dam**.

The study area corresponds to a regional DEM mosaic with a **lower-left corner at 50° N, 110° W** and extends to fully cover the contributing watershed upstream of Lake Diefenbaker. The resulting river network is approximately **five times denser than MERIT-Basins**, enabling finer-scale hydrologic analysis.

The DEM used for river extraction was **hydrologically conditioned**, and additional corrections were applied during preprocessing to ensure consistent drainage connectivity, add sinks, and enforce realistic flow paths. Both the derived river network and the associated subbasins were further processed to ensure compatibility with the *riverlakenetwork* package.

Subbasins were generated using **WhiteboxTools**; however, due to known issues related to raster-to-vector conversion (e.g., self-touching polygons, false interior holes, and topological artifacts), the resulting basin polygons **require additional post-processing**. These corrections were performed using supplementary scripts and, where necessary, external GIS tools such as **QGIS** to ensure valid geometries and correct basin topology prior to analysis.

**Citation**  
Yamazaki, D., Ikeshima, D., Sosa, J., Bates, P. D., Allen, G. H., & Pavelsky, T. M. (2019).  
*MERIT Hydro: A high-resolution global hydrography map based on the latest topography datasets.*  
**Water Resources Research**, 55, 5053–5073.  
https://doi.org/10.1029/2019WR024873

**Dataset Access**  
- MERIT Hydro global hydrography dataset:  
  https://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/


In [25]:
import numpy as np
import rasterio
from rasterio.transform import rowcol
from rasterio.features import rasterize
from shapely.geometry import LineString
from whitebox.whitebox_tools import WhiteboxTools
import matplotlib.pyplot as plt
import os
import sys
import glob
import shutil

# ============================================================
# INPUTS
# ============================================================

DEM_PATH = "/Users/shg096/Downloads/elv_n30w120/n50w110_elv.tif"

# Lines defined by start/end lon-lat
# Each entry: ((lon1, lat1), (lon2, lat2))
WALL_LINES_LON_LAT = [
    # example:
    ((-106.41960, 50.99760), (-106.44540, 50.96720)), # Qu'appelle dam
    ((-107.03700, 51.12200), (-107.01450, 51.09510)), # lucky lake
    ((-107.01450, 51.09510), (-107.04560, 51.05230)), # lucky lake
    ((-107.04560, 51.05230), (-107.06810, 51.04370)), # lucky lake
    ((-107.08960, 51.09840), (-107.09550, 51.03680)), # lucky lake split
    ((-106.87892, 51.25830), (-106.89287, 51.26091)), # gardiner spillway
    ((-106.77772, 51.14780), (-106.78822, 51.15736)), # Diefenbaker lake
    ((-107.61848, 50.66908), (-107.61762, 50.66406)), # Diefenbaker lake
    ((-108.06965, 50.67732), (-108.06556, 50.68268)), # Diefenbaker lake
    ((-108.06965, 50.67732), (-108.06556, 50.68268)), # Diefenbaker lake
    ((-107.61200, 50.66485), (-107.61182, 50.66427)), # Diefenbaker lake
    ((-108.06351, 50.68362), (-108.07337, 50.68503)), # Diefenbaker lake
    ((-108.07053, 50.68120), (-108.07028, 50.67732)), # Diefenbaker lake
    ((-106.71641, 51.05629), (-106.71628, 51.05257)), # Diefenbaker lake
    ((-106.66631, 51.12303), (-106.66238, 51.12312)), # Diefenbaker lake
]

WALL_HEIGHT = 50.0      # meters
BUFFER_CELLS = 2        # thickness of wall (cells on each side)
OUT_DEM = "dem_with_wall.tif"

for ext in ["*.shp", "*.shx", "*.dbf", "*.prj", "*.cpg", "*.tif", "*.gpkg"]:
    for f in glob.glob(os.path.join(os.getcwd(), ext)):
        try:
            os.remove(f)
            print(f"Deleted: {f}")
        except Exception as e:
            print(f"Failed to delete {f}: {e}")

# ============================================================
# READ DEM
# ============================================================

with rasterio.open(DEM_PATH) as src:
    dem = src.read(1)
    profile = src.profile
    transform = src.transform
    nodata = src.nodata
    height, width = dem.shape
    res_x = abs(transform.a)  # pixel size (deg for MERIT)

dem_out = dem.copy()

# ============================================================
# BUILD LINE GEOMETRIES (lon/lat CRS)
# ============================================================

lines = []
for (lon1, lat1), (lon2, lat2) in WALL_LINES_LON_LAT:
    lines.append(LineString([(lon1, lat1), (lon2, lat2)]))

if not lines:
    raise RuntimeError("❌ No wall lines provided")

# ============================================================
# BUFFER LINES (convert cells → map units)
# ============================================================

buffer_dist = BUFFER_CELLS * res_x
buffered_lines = [ln.buffer(buffer_dist) for ln in lines]

# ============================================================
# RASTERIZE WALL MASK
# ============================================================

wall_mask = rasterize(
    [(geom, 1) for geom in buffered_lines],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    dtype=np.uint8
)

# ============================================================
# APPLY WALL HEIGHT
# ============================================================

valid = (wall_mask == 1)
if nodata is not None:
    valid &= (dem_out != nodata)

dem_out[valid] += WALL_HEIGHT

print(f"Raised {valid.sum()} cells by {WALL_HEIGHT} m")

# ============================================================
# WRITE OUTPUT DEM
# ============================================================

with rasterio.open(OUT_DEM, "w", **profile) as dst:
    dst.write(dem_out, 1)

print(f"✅ DEM with wall written to: {OUT_DEM}")



Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/rivers_final_SK.shp
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/basins_final_SK.shp
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/rivers_final_SK.shx
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/basins_final_SK.shx
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/rivers_final_SK.dbf
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/basins_final_SK.dbf
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/basins_final_SK.prj
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/rivers_final_SK.prj
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/rivers_final_SK.cpg
Deleted: /Users/shg096/Desktop/RiverLakeNetwork/examples/preparation/basins_final_SK.cpg
Raised 1457 cells by 50.0 m
✅ DEM with wall written to: dem_with_wall.tif


In [26]:
# ============================================================
# USER INPUTS
# ============================================================

DEM_PATH = "dem_with_wall.tif" # from merit hydro

# sink point to alter the non contributing area when needed
SINK_POINTS_LONLAT = [
    (-107.06290, 51.07230),
    (-107.09460, 51.07610),
    (-107.50830, 50.85320),
    (-108.01120, 51.14725),
    (-108.08740, 51.09047),
    (-107.64240, 51.06080),
    (-108.43274, 50.96276),
    (-108.34569, 51.00249),
    (-108.52113, 51.24219),
    (-107.33030, 51.37085),
    (-106.34001, 51.14840),
    (-107.06583, 51.06914),
    
]

SINK_DEPTH = 20.0
FILL_DEPTH = 2.0
ACC_THRESHOLD = 1000

# ============================================================
# OUTPUT FILE NAMES (SAME FOLDER AS SCRIPT)
# ============================================================

DEM_SINKED = "dem_sinked.tif"
DEM_FILLED = "dem_filled.tif"
FLOW_DIR   = "flow_dir.tif"
FLOW_ACC   = "flow_acc.tif"

RIVERS_R   = "rivers.tif"
RIVERS_SHP = "rivers.shp"

POUR_PTS   = "pour_points.tif"
BASINS_R   = "basins.tif"
BASINS_SHP = "basins.shp"

# ============================================================
# SETUP WHITEBOX (CRITICAL)
# ============================================================

wbt = WhiteboxTools()
wbt.verbose = False
wbt.work_dir = os.getcwd()   # 🔑 required, but no directory handling needed

# ============================================================
# STEP 1 — READ DEM
# ============================================================

with rasterio.open(DEM_PATH) as src:
    dem = src.read(1)
    profile = src.profile
    transform = src.transform

dem_sinked = dem.copy()

# ============================================================
# STEP 2 — CONVERT LON/LAT → ROW/COL
# ============================================================

sink_rc = []
for lon, lat in SINK_POINTS_LONLAT:
    r, c = rowcol(transform, lon, lat)
    sink_rc.append((r, c))

print("Artificial sink cells (row, col):", sink_rc)

# ============================================================
# STEP 3 — BURN ARTIFICIAL SINKS
# ============================================================

for r, c in sink_rc:
    if 0 <= r < dem.shape[0] and 0 <= c < dem.shape[1]:
        dem_sinked[r, c] -= SINK_DEPTH

with rasterio.open(DEM_SINKED, "w", **profile) as dst:
    dst.write(dem_sinked, 1)

# ============================================================
# STEP 4 — FILL ONLY SMALL NATURAL DEPRESSIONS
# ============================================================

wbt.fill_depressions(
    DEM_SINKED,
    output=DEM_FILLED,
    max_depth=FILL_DEPTH
)

if not os.path.exists(DEM_FILLED):
    sys.exit("❌ fill_depressions failed")

# ============================================================
# STEP 5 — FLOW DIRECTION
# ============================================================

wbt.d8_pointer(
    DEM_FILLED,
    output=FLOW_DIR
)

if not os.path.exists(FLOW_DIR):
    sys.exit("❌ flow direction failed")

# ============================================================
# STEP 6 — FLOW ACCUMULATION
# ============================================================

wbt.d8_flow_accumulation(
    i=DEM_FILLED,
    output=FLOW_ACC,
    out_type="cells"
)

if not os.path.exists(FLOW_ACC):
    sys.exit("❌ flow accumulation failed")

# ============================================================
# STEP 7 — EXTRACT RIVER NETWORK
# ============================================================

wbt.extract_streams(
    flow_accum=FLOW_ACC,
    output=RIVERS_R,
    threshold=ACC_THRESHOLD
)

if not os.path.exists(RIVERS_R):
    sys.exit("❌ stream extraction failed")

# ============================================================
# STEP 8 — RIVERS → SHAPEFILE (FIXED)
# ============================================================

wbt.raster_streams_to_vector(
    streams=RIVERS_R,
    d8_pntr=FLOW_DIR,
    output=RIVERS_SHP
)

if not os.path.exists(RIVERS_SHP):
    sys.exit("❌ river vectorization failed")

# ============================================================
# STEP 9 — BASIN CREATION
# ============================================================
wbt.subbasins(
    d8_pntr=FLOW_DIR,
    streams=RIVERS_R,
    output=BASINS_R
)

if not os.path.exists(BASINS_R):
    sys.exit("❌ river vectorization failed")

# ============================================================
# STEP 10 — BASIN → SHAPEFILE (FIXED)
# ============================================================

# Convert subbasins to polygons
wbt.raster_to_vector_polygons(
    i=BASINS_R,
    output=BASINS_SHP
)

if not os.path.exists(BASINS_SHP):
    sys.exit("❌ river vectorization failed")

print("✅ ALL DONE — everything created in current folder")

Artificial sink cells (row, col): [(4712, 3525), (4708, 3486), (4975, 2990), (4622, 2387), (4690, 2295), (4726, 2829), (4844, 1881), (4796, 1985), (4508, 1775), (4354, 3204), (4621, 4392), (4716, 3521)]
✅ ALL DONE — everything created in current folder


# **Basins clean up**
## Clean up the invalid geometry in basins with QGIS  or other GIS programs and resave it to the same location before continuing the next steps
-------


-----
# **ID assigning to rivers and basins**
## In the following, the IDs of rivers and basins are assigned to each other so the associated river segment and basins have the same ID.

In [27]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon

# ============================================================
# FILES
# ============================================================

RIVERS_SHP = "rivers.shp"
BASINS_IN  = "basins.shp"
BASINS_OUT = "basins_with_linkno.shp"
RIVERS_OUT = "rivers_with_linkno.shp"
LINK_FIELD = "link_id" # link_id field in the rivers that should be assgined to basins

# ============================================================
# LOAD DATA
# ============================================================

rivers = gpd.read_file(RIVERS_SHP)
basins = gpd.read_file(BASINS_IN)

if LINK_FIELD not in rivers.columns:
    rivers[LINK_FIELD] = np.arange(1, len(rivers) + 1)

print(f"Loaded {len(rivers)} rivers")
print(f"Loaded {len(basins)} basins")

# ============================================================
# EXTRACT START POINT OF EACH RIVER
# ============================================================

records = []

for _, row in rivers.iterrows():
    geom = row.geometry

    if geom.geom_type == "LineString":
        coords = list(geom.coords)
    elif geom.geom_type == "MultiLineString":
        coords = list(geom.geoms[0].coords)
    else:
        raise TypeError("River geometry must be LineString or MultiLineString")

    records.append({
        LINK_FIELD: row[LINK_FIELD],
        "geometry": Point(coords[0])
    })

river_starts = gpd.GeoDataFrame(records, crs=rivers.crs)
river_starts.to_file("starting_points.shp")
river_starts["geometry"] = river_starts.geometry.buffer(0.00000001)

# ============================================================
# SPATIAL JOIN: BASIN ← RIVER START
# ============================================================

joined = gpd.sjoin(
    basins,
    river_starts,
    how="left",
    predicate="intersects"
)

# ============================================================
# REPORT & REMOVE UNMATCHED BASINS
# ============================================================

missing_mask = joined[LINK_FIELD].isna()
n_missing = missing_mask.sum()

if n_missing > 0:
    print(f"⚠️  Removing {n_missing} subbasins without a river start point")

# Keep only valid basins
basins_out = basins.loc[~missing_mask].copy()
basins_out[LINK_FIELD] = joined.loc[~missing_mask, LINK_FIELD].values

print(f"✅ Remaining basins: {len(basins_out)}")

# ============================================================
# SAVE OUTPUT
# ============================================================

rivers = rivers[rivers["link_id"].isin(set(basins_out["link_id"].astype(int)))]

basins_out.to_file(BASINS_OUT)
rivers.to_file(RIVERS_OUT)

print(f"✅ Output written: {BASINS_OUT}")

/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: basins.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Loaded 19910 rivers
Loaded 19923 basins


/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


⚠️  Removing 33 subbasins without a river start point
✅ Remaining basins: 19890


/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


✅ Output written: basins_with_linkno.shp


# **Assinging the next down ID in the river network**

In [28]:
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString, MultiLineString
from scipy.spatial import cKDTree

# ============================================================
# USER INPUTS
# ============================================================

RIVERS_SHP = "rivers_with_linkno.shp"  # input shapefile
ID_FIELD   = "link_id"        # <-- CHANGE THIS if needed
OUT_SHP    = "rivers_with_linkno_ds.shp"

DIST_TOL   = 1e-6             # distance tolerance (units of CRS)

# ============================================================
# LOAD RIVERS
# ============================================================

gdf = gpd.read_file(RIVERS_SHP)

if ID_FIELD not in gdf.columns:
    raise ValueError(f"ID field '{ID_FIELD}' not found")

print(f"Loaded {len(gdf)} river segments")

# ============================================================
# FUNCTIONS
# ============================================================

def get_start_end(geom):
    """Return (start_point, end_point) of a line geometry"""
    if isinstance(geom, LineString):
        coords = list(geom.coords)
    elif isinstance(geom, MultiLineString):
        coords = list(geom.geoms[0].coords)
    else:
        raise TypeError("Geometry must be LineString or MultiLineString")

    return np.array(coords[0]), np.array(coords[-1])

# ============================================================
# EXTRACT START / END POINTS
# ============================================================

starts = []
ends = []
ids = []

for _, row in gdf.iterrows():
    s, e = get_start_end(row.geometry)
    starts.append(s)
    ends.append(e)
    ids.append(row[ID_FIELD])

starts = np.array(starts)
ends   = np.array(ends)
ids    = np.array(ids)

# ============================================================
# BUILD KD-TREE ON START POINTS
# ============================================================

tree = cKDTree(starts)

# ============================================================
# FIND DOWNSTREAM SEGMENT
# ============================================================

down_ids = []

for i, end_pt in enumerate(ends):
    dist, idx = tree.query(end_pt, k=2)

    # idx[0] is usually self → check idx[1]
    candidate_idx = idx[1] if idx[0] == i else idx[0]
    candidate_dist = dist[1] if idx[0] == i else dist[0]

    if candidate_dist <= DIST_TOL:
        down_ids.append(ids[candidate_idx])
    else:
        down_ids.append(-9999)

# ============================================================
# ATTACH RESULTS
# ============================================================

gdf["link_id"] = ids
gdf["ds_link_id"] = down_ids

# ============================================================
# SAVE OUTPUT
# ============================================================

gdf.to_file(OUT_SHP)

print("✅ DONE")
print(f"Saved: {OUT_SHP}")


Loaded 19890 river segments


/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


✅ DONE
Saved: rivers_with_linkno_ds.shp


# **Assinging the unitarea and uparea**

In [29]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pyproj import CRS

# ============================================================
# INPUTS
# ============================================================

RIVERS_IN  = "rivers_with_linkno_ds.shp"
BASINS_IN  = "basins_with_linkno.shp"

RIVERS_OUT = "rivers_final_SK.shp"
BASINS_OUT = "basins_final_SK.shp"

LINK = "link_id"
DOWN = "ds_link_id"

# ============================================================
# LOAD DATA
# ============================================================

rivers = gpd.read_file(RIVERS_IN)
basins = gpd.read_file(BASINS_IN)

print(f"Loaded {len(rivers)} rivers")
print(f"Loaded {len(basins)} basins")

from pyproj import CRS

# ============================================================
# ENSURE CRS EXISTS
# ============================================================

if rivers.crs is None:
    print("⚠️ Rivers CRS missing → assigning EPSG:4326 (WGS84)")
    rivers = rivers.set_crs(epsg=4326)

if basins.crs is None:
    print("⚠️ Basins CRS missing → assigning EPSG:4326 (WGS84)")
    basins = basins.set_crs(epsg=4326)

# ============================================================
# CHOOSE METRIC CRS (GLOBAL EQUAL-AREA)
# ============================================================

metric_crs = CRS.from_epsg(6933)   # WGS 84 / NSIDC EASE-Grid 2.0 Global (equal-area)

print(f"Using metric CRS for calculations: {metric_crs.to_string()}")

# ============================================================
# TEMPORARY REPROJECT (METRICS ONLY)
# ============================================================

rivers_m = rivers.to_crs(metric_crs)
basins_m = basins.to_crs(metric_crs)

# ============================================================
# TEMPORARY REPROJECT (FOR METRICS ONLY)
# ============================================================

rivers_m = rivers.to_crs(metric_crs)
basins_m = basins.to_crs(metric_crs)

# ============================================================
# CALCULATE AREA & LENGTH
# ============================================================

basins["unitarea"] = basins_m.area / 1e6
rivers["length"]     = rivers_m.length / 1000.0

# ============================================================
# PASS UNIT AREA TO RIVERS
# ============================================================

area_map = dict(zip(basins[LINK], basins["unitarea"]))
rivers["unitarea"] = rivers[LINK].map(area_map)

# ============================================================
# UPSTREAM AREA CALCULATION
# ============================================================

# Build downstream → upstream graph
upstream = {lid: [] for lid in rivers[LINK]}

for _, r in rivers.iterrows():
    if r[DOWN] in upstream:
        upstream[r[DOWN]].append(r[LINK])

# Recursive accumulation
from functools import lru_cache

@lru_cache(None)
def compute_uparea(link):
    area = area_map.get(link, 0.0)
    for up in upstream.get(link, []):
        area += compute_uparea(up)
    return area

rivers["uparea"] = rivers[LINK].apply(compute_uparea)
basins["uparea"] = basins[LINK].apply(compute_uparea)

# ============================================================
# FINAL CLEANUP
# ============================================================

# Ensure numeric types
for col in ["unitarea", "uparea"]:
    basins[col] = basins[col].astype(float)
    rivers[col] = rivers[col].astype(float)

rivers["length"] = rivers["length"].astype(float)

# ============================================================
# SAVE (GEOMETRY UNCHANGED)
# ============================================================

rivers.to_file(RIVERS_OUT)
basins.to_file(BASINS_OUT)

print("✅ DONE")
print(f"Saved: {RIVERS_OUT}")
print(f"Saved: {BASINS_OUT}")


Loaded 19890 rivers
Loaded 19890 basins
⚠️ Rivers CRS missing → assigning EPSG:4326 (WGS84)
⚠️ Basins CRS missing → assigning EPSG:4326 (WGS84)
Using metric CRS for calculations: EPSG:6933
✅ DONE
Saved: rivers_final_SK.shp
Saved: basins_final_SK.shp


In [30]:
import os
import glob

# ============================================================
# FILES TO KEEP (without extensions)
# ============================================================
keep_files = ["rivers_final_SK", "basins_final_SK"]

# ============================================================
# SHAPEFILE-RELATED EXTENSIONS
# ============================================================
shp_exts = [".shp", ".shx", ".dbf", ".prj", ".cpg"]
raster_exts = [".tif", ".tiff"]

# ============================================================
# DELETE UNNEEDED FILES
# ============================================================

# Delete shapefiles not in keep_files
for f in glob.glob("*.shp"):
    basename = os.path.splitext(f)[0]
    if basename not in keep_files:
        for ext in shp_exts:
            target = f"{basename}{ext}"
            if os.path.exists(target):
                try:
                    os.remove(target)
                    print(f"Deleted: {target}")
                except Exception as e:
                    print(f"Failed to delete {target}: {e}")

# Delete all raster files except final DEM-derived outputs (if any)
for f in glob.glob("*.tif") + glob.glob("*.tiff"):
    if f not in []:  # leave empty or list specific files to keep
        try:
            os.remove(f)
            print(f"Deleted: {f}")
        except Exception as e:
            print(f"Failed to delete {f}: {e}")


Deleted: rivers_with_linkno_ds.shp
Deleted: rivers_with_linkno_ds.shx
Deleted: rivers_with_linkno_ds.dbf
Deleted: rivers_with_linkno_ds.cpg
Deleted: starting_points.shp
Deleted: starting_points.shx
Deleted: starting_points.dbf
Deleted: starting_points.cpg
Deleted: basins.shp
Deleted: basins.shx
Deleted: basins.dbf
Deleted: basins.prj
Deleted: basins_with_linkno.shp
Deleted: basins_with_linkno.shx
Deleted: basins_with_linkno.dbf
Deleted: basins_with_linkno.cpg
Deleted: rivers_with_linkno.shp
Deleted: rivers_with_linkno.shx
Deleted: rivers_with_linkno.dbf
Deleted: rivers_with_linkno.cpg
Deleted: rivers.shp
Deleted: rivers.shx
Deleted: rivers.dbf
Deleted: dem_with_wall.tif
Deleted: dem_filled.tif
Deleted: basins.tif
Deleted: rivers.tif
Deleted: flow_dir.tif
Deleted: flow_acc.tif
Deleted: dem_sinked.tif


In [31]:
asset_dir = "assets"
if os.path.exists(asset_dir):
    shutil.rmtree(asset_dir)
os.makedirs(asset_dir)
patterns = [
    "basins_final_SK.*",
    "rivers_final_SK.*",
]
for pattern in patterns:
    for file in glob.glob(pattern):
        shutil.move(file, os.path.join(asset_dir, os.path.basename(file)))

print("✅ asset folder refreshed and files moved successfully")

✅ asset folder refreshed and files moved successfully
